
At first we import the necessary librarys

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from warnings import filterwarnings
filterwarnings('ignore')

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Then, we load our dataset in df and take a look at its first 5 row of the data. It's a nice way to get an overview of the data.

In [ ]:
df = pd.read_csv('/kaggle/input/us-news-and-world-reports-college-data/College.csv',index_col=0)
df.head()

We can see that our dataset does not have NaN values

In [ ]:
df.info()

In [ ]:
df.describe()

Defining our function to convert categorical variables

In [ ]:
def converter(cluster):
    if cluster=='Yes':
        return 1
    else:
        return 0

Applying function converter

In [ ]:
df['Private'] = df['Private'].apply(converter)
df.info()

Defining our label and inputs

In [ ]:
X = df.drop(['Private'],axis=1)
y = df["Private"]

Importing methods to auxiliate in the choosing of our features

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif, mutual_info_classif

After running the model a couple times, we decided to use eleven features as a parameter to our feature classifier.

In [ ]:
f_classif = SelectKBest(score_func=f_classif, k=11)

In [ ]:
fit = f_classif.fit(X,y)

In [ ]:
features = fit.transform(X)
features

Getting the chosen features

In [ ]:
cols = fit.get_support(indices=True)
df.iloc[:,cols]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df.iloc[:,cols], y, test_size=0.25, random_state=42)


Let's preprocess our selected features to improve our model prediction 

In [ ]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler().fit(x_train)
x_scaled = scaler.transform(x_train)

kmeans = KMeans(n_clusters=2, random_state = 100)

kmeans.fit(x_scaled)

In [ ]:
scaler = preprocessing.StandardScaler().fit(x_test)
x_test_scaled = scaler.transform(x_test)

In [ ]:
y_pred = kmeans.predict(x_test_scaled)
print(classification_report(y_test,y_pred))

Using mae to check our model

In [ ]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(x_scaled, x_test_scaled, y_train, y_test):
    kmeans = KMeans(n_clusters=2, random_state = 100)
    kmeans.fit(x_scaled)
    preds = kmeans.predict(x_test_scaled)
    return mean_absolute_error(y_test, preds)

In [ ]:
print("MAE:")
print(score_dataset(x_scaled, x_test_scaled, y_train, y_test))